<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/CHAPTER4%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%91%9C%ED%98%84%EA%B3%BC%20%ED%8A%B9%EC%84%B1%20%EA%B3%B5%ED%95%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 들어가며

데이터가 2차원 실수형 배열로서 각 열이 샘플을 설명하면 연속형 특성(continuous feature)이라고 가정했습니다. 하지만 많은 애플리케이션에서는 이렇게 데이터가 수집되지 않습니다. 일반적인 특성의 전형적인 형태는 숫자 값이 아닌 범주형 특성(categorical feature)(=이산형 특성(discrete feature))입니다.

연속형 특성과 범주형 특성은 회귀와 분류의 차이와 비슷합니다. 다만 출력이 아닌 입력에 대한 것이란 점에서 차이가 있습니다. 앞서 살폈던 연속형 특성의 예는 픽셀 밝기, 붓꽃 측정값입니다. 범주형 특성의 예는 제품 브랜드, 색상, 판매 분류(책, 옷, 하드웨어) 등입니다. 이런 특성들은 모두 상품을 묘사하는 속성이므로 연속된 값이 아닙니다. 제품과 제품 간에는 중간값이 없고 순서도 없습니다.

하지만 데이터의 특성이 어떤 형태를 취하는지보다 중요한 것은 데이터를 표현하는 방식입니다. 이 방식에 따라 머신러닝 모델 성능에 주는 영향이 큽니다. 데이터의 스케일을 조정하는 것이 매우 중요하다는 점을 강조했습니다. 예를 들면 측정치의 스케일을 조정하지 않았을 때 센티미터, 인치의 기준에 따라 큰 차이가 생깁니다. 스케일 조정뿐만 아니라 특성의 상호작용(특성 간의 곱)이나 일반적인 다항식을 추가 특성으로 넣는 것도 방법입니다.

특정 애플리케이션에 가장 적합한 데이터 표현을 찾는 것을 '특성 공학(feature engineering)'이라고 합니다. 올바른 데이터 표현을 찾는다면 지도 학습 모델에서 적절한 매개변수를 선택하는 것보다 모델 성능을 더 크게 제고할 수 있습니다.

# 4.1 범주형 변수

예제에 사용할 데이터는 1994년 인구 조사 데이터베이스에서 추출한 미국 성인의 소득 데이터셋입니다. 

## 4.1.1 원-핫-인코딩(가변수)

### 범주형 데이터 문자열 확인하기

## 4.1.2 숫자로 표현된 범주형 특성

# 4.2 OneHotEncoder와 ColumnTransformer: scikit-learn으로 범주형 변수 다루기

# 4.3 make_column_transformer로 간편하게 ColumnTransformer 만들기

# 4,4 구간 분할, 이산화 그리고 선형 모델, 트리 모델

# 4.5 상호작용과 다항식

# 4.6 일변량 비선형 변환

# 4.7 특성 자동 선택

## 4.7.1 일변량 통계

## 4.7.2 모델 기반 특성 선택

## 4.7.3 반복적 특성 선택

# 4.8 전문가 지식 활용

# 4.9 요약 및 정리